# Generating Beading Patterns 

This notebook implements visualizing procedurally generated beading patterns that are used to train our flow matching model.

In [ ]:
import matplotlib.pyplot as plt
from plate_optim.pattern_generation import BeadingTransition, draw_high_variation_img
from plate_optim.utils.plotting import plot_contraints_violations
import matplotlib.pyplot as plt
import numpy as np
from plate_optim.metrics.manufacturing import calc_beading_ratio,mean_valid_pixels
from plate_optim.project_directories import main_dir
import os

### Draw images used for D50000 dataset

In [ ]:
MIN_LENGTH_SCALE = 0.025

In [ ]:
# Define the beading geometry
eng_beading = BeadingTransition(h_bead= 0.02, r_f = 0.0095, r_h = 0.0095, alpha_F= 70*np.pi/180)

fig, ax = plt.subplots(4,4,figsize=(12,12))
for ax in ax.flat:
    img = draw_high_variation_img(dimension= np.array([0.9, 0.6]), 
                                resolution= np.array([181, 121]), 
                                height = 0.02, 
                                n_lines = [1,2], 
                                n_snakes= [0,1], 
                                n_rect = [0,2], 
                                n_arc= [0,2], 
                                sym = "partial",  
                                eng_beading = eng_beading,
                                max_beading_ratio= 0.5)                                  
    #ax.imshow(img, cmap='gray')
    plot_contraints_violations(img,ax=ax,min_length_scale=MIN_LENGTH_SCALE)
    ax.axis("off")



# Cross section of last beading at x = 0.45
fig, ax = plt.subplots()
ax.plot(np.linspace(0.0, 0.6, 121), img[:,90])
ax.axis("equal")
plt.tight_layout()

plt.show()


### Draw images with minimum length scale

In [ ]:
# Define the beading geometry
eng_beading = BeadingTransition(h_bead= 0.02, r_f = 0.0095, r_h = 0.0095, alpha_F= 70*np.pi/180)

fig, ax = plt.subplots(4,4,figsize=(12,12))
for ax in ax.flat:
    img = draw_high_variation_img(dimension= np.array([0.9, 0.6]), 
                                resolution= np.array([181, 121]), 
                                height = 0.02, 
                                n_lines = [1,2], 
                                n_snakes= [0,1], 
                                n_rect = [0,2], 
                                n_arc= [0,2], 
                                sym = "partial",  
                                eng_beading = eng_beading,
                                max_beading_ratio= 0.5,
                                min_length_scale=MIN_LENGTH_SCALE)                                  
    

    plot_contraints_violations(img,ax=ax,min_length_scale=MIN_LENGTH_SCALE)
    beading_ratio = img.sum()/(img.size*img.max())
    ax.set_title(f"{beading_ratio:.2f}")

# Cross section of last beading at x = 0.45
fig, ax = plt.subplots()
ax.plot(np.linspace(0.0, 0.6, 121), img[:,90])
ax.axis("equal")

plt.show()



### Comparison

In [ ]:
import numpy as np
np.array((3,4))*45

In [ ]:
fig, ax = plt.subplots(4,4,figsize=(10.5, 8))
import torch
for ax in ax.flat:
    img = draw_high_variation_img(dimension= np.array([0.9, 0.6]), 
                                resolution= np.array([181, 121]), 
                                height = 0.02, 
                                n_lines = [1,2], 
                                n_snakes= [0,1], 
                                n_rect = [0,2], 
                                n_arc= [0,2], 
                                sym = "partial",  
                                eng_beading = eng_beading,
                                max_beading_ratio= 0.5,
                                min_length_scale=MIN_LENGTH_SCALE)           
    
    # convert img to torch tensor and interpolate
    img = torch.tensor(img)
    img = torch.nn.functional.interpolate(img.unsqueeze(0).unsqueeze(0), size=(96, 128), mode='bilinear', align_corners=True).squeeze(0).squeeze(0).numpy()
    ax.imshow(img, cmap='gray')
    ax.axis('off')     
    # print(img.shape)  
plt.tight_layout()
plt.savefig(os.path.join(main_dir, 'plots', 'dataset_samples.pdf'), bbox_inches='tight', pad_inches=0.0, dpi=300)                
    

In [ ]:
imgs_kernel = [
    draw_high_variation_img(dimension= np.array([0.9, 0.6]), 
                                resolution= np.array([181, 121]), 
                                height = 0.02, 
                                n_lines = [1,2], 
                                n_snakes= [0,1], 
                                n_rect = [0,2], 
                                n_arc= [0,2], 
                                sym = "partial",  
                                eng_beading = eng_beading,
                                max_beading_ratio= 0.5,
                                min_length_scale=MIN_LENGTH_SCALE)                                  
    for i in range(500)
]

In [ ]:
imgs_no_kernel = [
    draw_high_variation_img(dimension= np.array([0.9, 0.6]), 
                                resolution= np.array([181, 121]), 
                                height = 0.02, 
                                n_lines = [1,2], 
                                n_snakes= [0,1], 
                                n_rect = [0,2], 
                                n_arc= [0,2], 
                                sym = "partial",  
                                eng_beading = eng_beading,
                                max_beading_ratio= 0.5,
                                min_length_scale=0.0)                                  
    for i in range(500)
]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,12))
plot_contraints_violations(imgs_kernel[0],ax=ax)

In [ ]:
imgs_kernel_ratios = [
    calc_beading_ratio(img)
    for img in imgs_kernel
]

imgs_no_kernel_ratios = [
    calc_beading_ratio(img)
    for img in imgs_no_kernel
]
plt.title("Beading ratio")
plt.hist(imgs_kernel_ratios,histtype="step",label="with kernel")
plt.hist(imgs_no_kernel_ratios,histtype="step",label="no kernel")
plt.legend()
plt.show()

In [ ]:
print("With kernel:")

print("Valid pixels:",mean_valid_pixels(imgs_kernel,min_beading_size=MIN_LENGTH_SCALE, min_beading_space=MIN_LENGTH_SCALE))


print("Without kernel:")

print("Valid pixels:",mean_valid_pixels(imgs_no_kernel,min_beading_size=MIN_LENGTH_SCALE,min_beading_space=MIN_LENGTH_SCALE))


In [ ]:
imgs_kernel = [
    draw_high_variation_img(dimension= np.array([0.9, 0.6]), 
                                resolution= np.array([181, 121]), 
                                height = 0.02, 
                                n_lines = [1,2], 
                                n_snakes= [0,1], 
                                n_rect = [0,2], 
                                n_arc= [0,2], 
                                sym = "partial",  
                                eng_beading = eng_beading,
                                max_beading_ratio= 0.5,
                                min_length_scale=MIN_LENGTH_SCALE)                                  
    for i in range(500)
]

print(np.mean(imgs_kernel), np.std(imgs_kernel))